In [1]:
#====================================================================================================
# kogpt2 를 이용해 훈련한 모델 추론(생성) 요약(abstractive summarization) 훈련 예시
# => 
# => https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/
#
# => text generation 모델이므로  acc 구하는 것은 의미 없음(*따라서 train loss, val loss 만 구함)
#
# => 여기서는 훈련할때 요약할 문장과 요약 문장사이에 구분자 토큰을 <segment>로 지정하였음.(해당 토큰은 다른것으로 지정하여 훈련시켜도 됨)
# [훈련 dataset]
# => input_ids = '요약할 문장<segment>요약문</s>'    : <segment>토큰은 요약할 문장과 요약문 사이 구분자로 대체 가능함
# => labels = input_ids와 동일
#
# [추론(생성) 요약(abstractive summarization) 훈련 과정]
# 
# 1. gpt-2 모델 선언(GPT2LMHeadModel), tokenizer 선언(PreTrainedTokenizerFast)
# 2. '요약할 문장+구분token(<summarize>)+요약문+<eos>토큰' 식으로 된 훈련 dataset 생성
# 3. 모델에 input_ids, lables 을 입력하여 훈련 시킴
#====================================================================================================

import torch
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler 
import numpy as np
import pandas as pd
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AutoModelForCausalLM
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm.notebook import tqdm
import os
import time
from myutils import GPU_info, seed_everything, mlogging, SaveBERTModel, AccuracyForMLM
from summarizer import TransformerSummarizer
model_path='../model/gpt-2/kakao-kogpt-float16/'
#model_path='skt/kogpt2-base-v2'
#model_path = "gpt2-medium"

# 출력
OUTPATH = '../model/gpt-2/kakao-kogpt-float16-ft-sum'

device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="gpt2-ft", logfilename="../log/gpt2-ft")

logfilepath:../log/bwdataset_2022-05-30.log
logfilepath:../log/qnadataset_2022-05-30.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/gpt2-ft_2022-05-30.log


In [2]:
# 요약문 corpus 파일 열기 
corpus_path = "../korpora/newspaper.csv"
with open(corpus_path, "r") as reviews_raw:
    reviews = reviews_raw.readlines()

In [3]:
reviews[:3]

['올여름 장마가 17일 제주도에서 시작됐다  서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다  제주의 장마는 평년보다 2 3일  지난해보다는 하루 일찍 시작됐다  장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다  이에 따라 20 21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다  그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다  장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다  최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24 25일이었으며 장마기간은 32일  강수일수는 17 2일이었다 기상청은 올해 장마기간의 평균 강수량이 350 400㎜로 평년과 비슷하거나 적을 것으로 내다봤다  브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다 ,제주도 장마 시작   중부는 이달 말부터\n',
 '부산시와  재 부산정보산업진흥원 원장 이인숙 이  2020 2021년 지역SW서비스사업화 지원사업  공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다  과학기술정보통신부가 주관하는  지역SW서비스사업화 지원사업 은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다  올해부터 2개년으로 진행되며  국비와 시비  민자 등 2년간 약 

In [4]:
print(len(reviews))

29811


In [5]:
# reviews 문장은 text, 요약문 식으로 되어 있다.
# => 이 구분자 , 를 <summarize> 로 변경함.(*<summarize> 토큰이 아니라, 다른 토큰을 지정해도 됨)
reviews = [review.replace(",", "<summarize>") for review in reviews]

In [6]:
reviews[1]

'부산시와  재 부산정보산업진흥원 원장 이인숙 이  2020 2021년 지역SW서비스사업화 지원사업  공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다  과학기술정보통신부가 주관하는  지역SW서비스사업화 지원사업 은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다  올해부터 2개년으로 진행되며  국비와 시비  민자 등 2년간 약 37억원의 예산이 투입된다  앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양  지능형기계  지능정보서비스 분야로 사전 수요조사를 진행했고  평가를 통해 선정된 5개 과제를 공모사업에 신청했다  그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다  당 사업은 전국 진흥기관을 대상으로 공모를 시작해  총 17개 지역에서 42개 과제가 선정되었으며  4개 과제가 선정된 곳은 부산과 강원지역 뿐이다  금번 선정된 과제들은  인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발  등 총 4개 과제다  부산시가 지원하고  부산정보산업진흥원과 지역기업  대학  연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다  2개의 Track으로 구분되는 이번사업은 Track 1 SW중소기업 에서 ㈜에이아이플랫폼  엔컴 주   Track 2 스타트업 에서는 ㈜토즈  삼보테크놀로지를 지원한다  ○  Track 1 의  주 에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질 아밀로이드 플라크 을 자체개발 관측장비로 진단한다  이를 통해 치매를 조기 발견하여  각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리체계 개발을 목표로 한다  ○ 엔컴 주 이 주관기업으로 참여하는 <AI영상분석 기반 가공철근 생산성 향상 시스템 기술개발 및 사업화>는 산업안전 

In [7]:
# 총 단어의 길이를 얻어옴
avg_length = sum([len(review.split()) for review in reviews])/len(reviews)
avg_length

99.39569286504981

In [8]:
# 총 단어의 길이보다 길게 설정
max_length = 100

In [9]:
# tokenizer 로딩 
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path,
                                                   bos_token='</s>',
                                                   eos_token='</s>',
                                                   unk_token='<unk>',
                                                   pad_token='<pad>',
                                                   mask_token='<mask>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# 모델 정의 하고, embedding size를 tokenizer 사이즈만큼 조정
#model = GPT2LMHeadModel.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
  model_path, revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device=device, non_blocking=True)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(64002, 4096)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=

In [11]:
# ** 구분자  <summarize> 토큰의 길이를 얻어옴
print(tokenizer.encode("<summarize>"))
extra_length = len(tokenizer.encode("<summarize>"))
print(f'***<summarize> token len:{extra_length}')


[134, 189, 2076, 19709, 15208, 136]
***<summarize> token len:6


In [12]:
# dataset 설정 함수
class summarizeDataset(Dataset):
    def __init__(self, tokenizer, sentences, max_len, summarize_token_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.sentences = sentences
        self.summarize_token_len = summarize_token_len
        self.result = []
        
        for sentence in self.sentences:
            # 한 문장 뒤에 </s>(EOS 토큰) 추가
            tokenized = self.tokenizer.encode(sentence + self.eos)
            #print(tokenized)
            # padd 
            padded = self.pad_truncate(tokenized)
            
            # 출력
            self.result.append(torch.tensor(padded))
           
    def __len__(self):
        return len(self.result)
    
    def __getitem__(self, item):
        return self.result[item]
    
    # padd 붙이는 함수
    def pad_truncate(self, name):
        
        # name_length는 총 name 길이에서 - summzrize_token 길이를 뺀 길이가 됨
        # (예: name 길이 = 110 이면 name_length = 110 - 8 = 102)
        name_length = len(name) - self.summarize_token_len
        
        # name 길이 < 100 작으면, 뒤에 108개까지는 eos_id(1)로 padd 붙임
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        # name 길이 > 100 크면, 100+7 까지만 name 값 출력하고, 뒤에 eos_id(1) 붙임 = 총 108개가 됨
        elif name_length > self.max_len:
            result = name[:self.max_len + self.summarize_token_len - 1]+[self.eos_id]
        else:
            result = name
        
        return result
        

In [13]:
# dataset 만듬
dataset = summarizeDataset(tokenizer, reviews, max_length, extra_length)

In [14]:
dataset[1]

tensor([34983,   533,   327,   871,  2060,  2040,  2401,  8143,   505,  3984,
        16670,  1807,   410,   327,  3248,  5564,   515,  1189, 38950,  3438,
         1781,   546,  1471,  1781,   327,  6563,  1781,   379,   698,   702,
         4307,   402,  2512,   540,   448,  3677, 59061,  1498,  8964,   379,
        15547,   327,  2578,  2551,  2040,  4611,  6655,  4787,   385,   378,
          327,  1189, 38950,  3438,  1781,   546,  1471,  1781,  1090, 32471,
        38950,  1986,   787,  2679,  9508,  3671, 18657,  3438,  1498,   546,
         1471,   450,   678,  2049,  3270,  1471,   387,  1132,  1640,  5509,
         2033,   533,  1189,  2511,  2812,   546,   413,  5445,   385,   378,
         1498, 35761,   327,  2620,   879,   479, 20961,   447,  1395,   540,
          566,   327, 12905,   533,  9957, 63998])

In [15]:
# 데이터 로더 생성 
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [16]:
# 훈련 시작 

##################################################
epochs = 2            # epochs
learning_rate = 3e-5  # 학습률
##################################################

# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs
warmup_steps = total_steps * 0.1 #10% of train data for warm-up

# 손실률 보여줄 step 수
p_itr = int(len(train_loader)*0.1)  
    
# step마다 모델 저장
save_steps = int(total_steps * 0.5)
    
# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=warmup_steps, 
                                            num_training_steps=total_steps)

itr = 1

total_loss = 0
list_train_loss = []

# 그래디언트 초기화(*set_to_none=True 로 설정하면, 그래디언트 업데이트시, 쓰기작업만 수행되어 속도가 빨라진다)
model.zero_grad(set_to_none=True)
for epoch in tqdm(range(epochs)):

    model.train() # 훈련모드로 변환
    for data in tqdm(train_loader):
        model.zero_grad(set_to_none=True)# 그래디언트 초기화(*set_to_none=True 로 설정하면, 그래디언트 업데이트시, 쓰기작업만 수행되어 속도가 빨라진다)
        
        # 입력 값 설정
        input_ids = data.to(device)
        labels = data.to(device)
        #print('Labels:{}'.format(labels))
        
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                        labels=labels)
        
       
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits
        #print('Loss:{}, logits:{}'.format(loss, logits))
        
        # logits_shape: torch.Size([32, 68, 51200])
        # => batch_size, sequence_max_len, token_len
        #print(f'logits_shape: {logits.shape}')                    
        
        # optimizer 과 scheduler 업데이트 시킴
        loss.backward()   # backward 구함
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
        optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
        scheduler.step()  # 학습률 감소
        
        # ***further pretrain 에는 손실률 계산을 넣지 않음
        # 정확도 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            
            # 손실률 계산
            total_loss += loss.item()
                
            #===========================================
            # 정확도(Accurarcy) 계산
            #correct = AccuracyForMLM(logits, labels, attention_mask)           
            #total_correct += correct.sum().item() 
            #total_len += attention_mask.sum().item()
            #=========================================
     
            # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
            if itr % p_itr == 0:
                
                train_loss = total_loss/p_itr
                                   
                logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}'.format(epoch+1, epochs, itr, train_loss))
                     
                list_train_loss.append(train_loss)
                 
                # 변수들 초기화    
                total_loss = 0
                ####################################################################
            if itr % save_steps == 0:
                #전체모델 저장
                SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

        itr+=1

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/931 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 106.00 MiB (GPU 0; 23.69 GiB total capacity; 21.70 GiB already allocated; 40.19 MiB free; 21.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 모델 저장
SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_train_loss, label='Train Loss')
#plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()


In [ ]:
# 추론시 topk 알고리즘 사용
def topk(probs, n=9):
    probs = torch.softmax(probs, dim=-1)
    
    tokensProb, topIx = torch.topk(probs, k=n)
    tokensProb = tokensProb / torch.sum(tokensProb)
    
    tokensProb = tokensProb.cpu().detach().numpy()
    
    choice = np.random.choice(n,1,p=tokensProb)
    tokenId = topIx[choice][0]
    
    return int(tokenId)

In [ ]:
# summarize 추론 처리 함수 
def model_infer(model, tokenizer, review, max_length=15):
    review_encoded = tokenizer.encode(review)
    result = review_encoded
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to(device)
    
    with torch.set_grad_enabled(False):
        outputs = model(initial_input)
        
        logits = outputs.logits[0,-1]
        print(logits.shape)  # embedding 계수 출력됨
        #result.append(topk(logits))
        #print(result)
        
        for _ in range(max_length):
            input = torch.tensor(result).unsqueeze(0).to(device)
            outputs = model(input)
            logits = outputs.logits[0,-1]
            res_id = topk(logits)
            
            if res_id == tokenizer.eos_token_id:
                return tokenizer.decode(result)
            else:
                result.append(res_id)
                
    return tokenizer.decode(result)
            

In [ ]:
# A review is initially fed to the model.
# A choice from the top-k choices is selected.
# The choice is added to the summary and the current sequence is fed to the model.
# Repeat steps 2 and 3 until either max_len is achieved or the EOS token is generated.

body = '''
전남도는 청정하고 안전한 신재생에너지 보급 확산을 위해 추진하는 ‘2022년 신재생에너지 보급 주택지원사업’ 참여 가구를 9일부터 6월 3일까지 모집한다고 밝혔다.
신재생에너지 보급 주택지원사업은 주택에 태양광, 태양열, 연료전지, 지열 등 신재생에너지 설비를 설치하는 도민에게 정부 지원금 외에 도비와 시군비를 추가로 지원하는 사업이다.
사업 대상자로 선정되면 설치비 자부담분(50%)의 40%를 도비와 시군비로 지원하기 때문에 비용부담을 줄일 수 있다.
전남도는 올해 지방비 17억원을 들여 1600여 가구에 신재생에너지 설비를 보급할 계획이다.
지원을 바라는 주택 소유자는 한국에너지공단 그린홈 누리집에서 회원가입 후 공단에 등록한 참여업체를 지정해 신청하면 된다.
지방비 보조금은 한국에너지공단의 최종 사업 승인 후 해당 시군에 지원 신청을 하면 예산 범위에서 선착순 지원한다.
자세한 사항은 한국에너지공단 광주전남지역본부, 전남도 에너지신산업과, 시군 에너지업무 담당 부서로 문의하면 된다.
주택에 3KW 태양광 설비를 설치하면 총 설치비 516만원 중 국비 258만원과 추가로 지방비 103만원을 지원하기 때문에 155만원만 자부담하면 된다.
3KW 태양광 설비를 설치한 가구는 월 4만5000원씩 연간 54만원의 전기요금을 절감할 수 있다.
'''

output = model_infer(model, tokenizer, body + "<summarize>", 100)
#print(output)

summary = output.split("<summarize>")[1].strip()
print(summary)